## Prepare for Transfer Learning (Data Augmentation)

In [1]:
import torch
from torch import nn
import os
import torchvision
from torchvision.datasets import ImageFolder
from torchvision import transforms
from torch.utils.data import DataLoader

NUM_BATCHES = 256
NUM_WORKERS = os.cpu_count()

data_transforms = {
    'train': transforms.Compose([
        transforms.Resize(size=(64, 64)),
        transforms.RandomHorizontalFlip(),
        transforms.RandomVerticalFlip(),
        transforms.RandomCrop(54),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], # ImageNet's data values used for the Pre-Trained_Model(ResNet50)
                             [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.Resize(size=(64, 64)),
        transforms.RandomCrop(54),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406],
                             [0.229, 0.224, 0.225])
    ]),
    'test': transforms.Compose([
        transforms.Resize(size=(64, 64)),
        transforms.ToTensor(),
    ])
}

base_dir = './splitted_data'

img_datasets = {x: ImageFolder(root=os.path.join(base_dir, x),
                                    transform=data_transforms[x]) for x in ['train', 'val', 'test']}
dataloaders = {x: torch.utils.data.DataLoader(dataset=img_datasets[x], 
                                              batch_size=NUM_BATCHES, num_workers=4, 
                                              shuffle=True) for x in ['train', 'val', 'test']}

class_names = img_datasets['train'].classes

## Import the Pre-Trained Model

In [2]:
#DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
DEVICE = "cpu"

In [3]:
from torchvision import models
import torch.optim
from torch.optim import lr_scheduler

# import the pretrained ResNet50 and change the fc layer to the apppropriate output_features (34 classes)
resnet = models.resnet50(pretrained=True)
num_ftrs = resnet.fc.in_features
resnet.fc = nn.Linear(num_ftrs, 34)
resnet = resnet.to(DEVICE)

# Define the loss function and optimizer
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(params=filter(lambda p: p.requires_grad, resnet.parameters()),
                             lr = 0.001)

# lower the learning rate by multiplying 0.1 every 7 epochs
res_lr_scheduler = lr_scheduler.StepLR(optimizer,
                                       step_size=7, gamma=0.1)

C:\Users\Admin\AppData\Local\Programs\Python\Python311\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\Users\Admin\AppData\Local\Programs\Python\Python311\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


## Freeze some parts of Pre-Trained Model's Layer

In [4]:
layer = 0
for child in resnet.children():
    layer += 1
    if layer < 6:
        for param in child.parameters():
            param.requires_grad = False

In [31]:
import time
import copy

dataset_sizes = {x: len(img_datasets[x]) for x in ['train', 'val', 'test']}

def train_resnet(model: torch.nn.Module,
                 loss_fn: torch.nn.Module,
                 optimizer: torch.optim.Optimizer,
                 scheduler,
                 num_epochs: int = 25):
    
    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0
    
    for epoch in range(num_epochs):
        print(f"-------- epoch {epoch} --------")
        start_time = time.time()
        
        for step in ['train', 'val']:
            if step == 'train':
                model.train()
            else:
                model.eval()
            
            running_loss = 0.0
            running_corrects = 0
            
            for inputs, labels in dataloaders[step]:
                inputs, labels = inputs.to(DEVICE), labels.to(DEVICE)
                
                optimizer.zero_grad()
                
                with torch.set_grad_enabled(step == "train"):
                    outputs = model(inputs)
                    preds = torch.argmax(outputs, dim=1)
                    loss = loss_fn(outputs, labels)
                    
                    if step == 'train':
                        loss.backward()
                        optimizer.step()
                        
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)
            
            if step == 'train':
                scheduler.step()
                l_r = [x['lr'] for x in optimizer.param_groups]
                print("learning rate: ", l_r)
                
            epoch_loss = running_loss/dataset_sizes[step]
            epoch_acc = running_corrects.double()/dataset_sizes[step]
            
            print(f"{step} Loss: {epoch_loss:.4f} | Acc: {epoch_acc}")
            
            if step == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())
                
        end_time = time.time()
        time_elapsed = end_time - start_time
        print(f"Completed in {time_elapsed / 60:.0f}min {time_elapsed % 60:0f}s")
    print(f"Best val Acc: {best_acc:.4f}")
    
    model.load_state_dict(best_model_wts)
    
    return model

## Train the ResNet50 Model

In [24]:
torch.cuda.empty_cache()

In [25]:
!nvidia-smi

Fri Dec 15 18:50:05 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 512.78       Driver Version: 512.78       CUDA Version: 11.6     |
|-------------------------------+----------------------+----------------------+
| GPU  Name            TCC/WDDM | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ... WDDM  | 00000000:01:00.0 Off |                  N/A |
| N/A   40C    P8    N/A /  N/A |   1979MiB /  2048MiB |      1%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [32]:
model_resnet50 = train_resnet(resnet, loss_fn, optimizer, res_lr_scheduler, num_epochs = 19)

-------- epoch 0 --------
learning rate:  [0.001]
train Loss: 0.5387 | Acc: 0.8432670190872039
val Loss: 0.3344 | Acc: 0.9076699382042893
Completed in 12min 26.150737s
-------- epoch 1 --------
learning rate:  [0.001]
train Loss: 0.1910 | Acc: 0.941769904362213
val Loss: 0.1229 | Acc: 0.9601357082273113
Completed in 13min 19.335580s
-------- epoch 2 --------
learning rate:  [0.001]
train Loss: 0.1393 | Acc: 0.9562164561559259
val Loss: 0.1073 | Acc: 0.9635284139100933
Completed in 13min 10.449545s
-------- epoch 3 --------
learning rate:  [0.001]
train Loss: 0.1141 | Acc: 0.9631976110729995
val Loss: 0.0629 | Acc: 0.9804919423240034
Completed in 12min 11.454653s
-------- epoch 4 --------
learning rate:  [0.001]
train Loss: 0.0903 | Acc: 0.9715104313788789
val Loss: 0.0675 | Acc: 0.9778262450018175
Completed in 12min 0.405803s
-------- epoch 5 --------
learning rate:  [0.001]
train Loss: 0.0743 | Acc: 0.9770388604172552
val Loss: 0.0705 | Acc: 0.9778262450018175
Completed in 13min 49.42

KeyboardInterrupt: 

## Save the Model Parameters

In [33]:
torch.save(model_resnet50, 'resnet50.pt')

NameError: name 'model_resnet50' is not defined

## Evaluate Transfer Learning Model(ResNet50) on the test dataset

In [34]:
def evaluate(model: torch.nn.Module,
             dataloader: torch.utils.data.DataLoader,
             optimizer: torch.optim.Optimizer, 
             loss_fn: torch.nn.Module=nn.CrossEntropyLoss()):
    # Put the model into evaluation mode
    model.eval()
    test_loss, correct, test_acc = 0, 0, 0
    
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(DEVICE), y.to(DEVICE)
            # Forward pass
            pred_probs = model(X)
            # Accumulate the loss
            test_loss += loss_fn(pred_probs, y).item()
            
            pred = torch.argmax(pred_probs, dim=1)
            correct += pred.eq(y.view_as(pred)).sum().item()
    
    test_loss /= len(dataloader)
    test_acc = 100. * correct / len(dataloader.dataset)
    return test_loss, test_acc

In [35]:
resnet50.eval()
test_loss, test_acc = evaluate(resnet50, test_dataloader, optimizer, loss_fn)

print(f"ResNet test acc: {test_acc}")

NameError: name 'resnet50' is not defined